In [1]:
import pandas as pd
import numpy as np
import os
import  matplotlib.pyplot as plt
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

In [2]:
country  = 'ethiopia'
err_type = 'scaled_mse'
scale    = 1
# Directory after running predict-v3.ipynb.
path_to_results = '../outputs_v23_prod7_crop_sales/'
path_to_tables  = '../Tables_best7_v22/'

In [3]:
outcomes = sorted(set([x.split('___')[0] for x in os.listdir(path_to_results) if 'coef' not in x and 'clus' not in x and 'plot' not in x and 'lock' not in x and 'elbow' not in x]))

model_names = {
# 'All variables':'All Inputs',
'Highly correlated policy':'Policy Relevant Inputs',
'Highly correlated all':'All relvent Inputs',
# 'Policy variables':'Policy Inputs',
# 'All without correlations': 'All without correlations',
}

index_row = []

In [4]:
outcomes

['crop_sales']

In [5]:

def summary_mse(year,outcome,var_spec):
    
    mse = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==False)][err_type].values[0]

    idx = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==True)].index#[err_type].idxmin()

    mse_segments = list(raw_mse.loc[idx][err_type])

    segments = list(raw_mse.loc[idx]['segment'].apply(lambda x: x.replace('___' + country + '_'+str(year), '')))

    all_series = pd.DataFrame()
    for segment, mse_segment in zip(segments, mse_segments):
        series = pd.Series([
            year,
            outcome.replace('_',' ').title(),
            'LASSO + '+model_names[var_spec],
            np.round(mse/scale,3),
            segment.replace('_',' ').title(),
            np.round(mse_segment/scale,3),
            np.round(mse/mse_segment-1,3)],
            index=[
            'Year',
            'Outcome',
            'Model',
            'Mean Sq. Error',
            'Household Segmentation',
            'Mean Sq. Error Segmented',
            '% Reduction in Error',
            ])
        all_series = pd.concat([all_series, series], 1)
    
    return all_series
        
            
def summary_coef(year,outcome,var_spec,is_clustered=True):
    global index_row
    # Get Segment Corresponding to Min MSE
    idx = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==is_clustered)].index#[err_type].idxmin()

    best_segments = list(raw_mse.loc[idx]['segment'].apply(lambda x: x.replace('___' + country + '_'+str(year), '')))
    all_series = pd.DataFrame()
    for best_segment in best_segments:
#         &(raw_coef['segmentation']==var_spec[:-2]
        best_var_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
            regex='policy').iloc[0].astype(float).apply(np.abs).idxmax()

#     &(raw_coef['segmentation']==var_spec[:-2])
        best_coef_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
            regex='policy').iloc[0].astype(float)[best_var_all]
        segments = int(math.pow(2, len(best_segment.split(','))))
        best_var = {}
        best_coef = {}
        best_of = 5
        for k in range(4):
#             try:
            best_c = raw_coef.loc[
            (raw_coef['inputs']==var_spec)&\
            (raw_coef['segmentation']==best_segment)&\
            (raw_coef['segment']==str(k))].filter(
            regex='policy').iloc[0].astype(float).apply(np.abs).nlargest(best_of)
#             best_t = best_c.apply(lambda x: x.index.tolist(), axis=1)
#             print (best_t.as_matrix())
#             best_coefs =  best_c.as_matrix()
            best_vars = best_c.index.tolist()
            best_coefs = raw_coef.loc[
            (raw_coef['inputs']==var_spec)&\
            (raw_coef['segmentation']==best_segment)&\
            (raw_coef['segment']==str(k))].iloc[0][best_vars].as_matrix()
            print (best_coefs)

            row = []
            index_row =[]
            for ix, t, i in zip(range(best_of), best_vars, best_coefs):
                row += [
                    t.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
                    np.round(i,3)
                ]
                index_row += ['Var ' + str(ix), 'Coef ' +  str(ix)]
#             except:
#                 print (var_spec, best_segment, k)
            series = pd.Series([
                year,
                outcome.replace('_',' ').title(),
                'LASSO + '+model_names[var_spec],
                best_var_all.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
                np.round(best_coef_all,3),
                best_segment.replace('___ethiopia_'+str(year), '').title()] + row,index=[
                'Year',
                'Outcome',
                'Model',
                'Most Relevant Input',
                'Coef. Most Relevant Input',
                'Household Segmentation']
                    + index_row)
            all_series = pd.concat([all_series, series], 1)
        
    return all_series

def summary_coef_unsegmented(year,outcome,var_spec):
    global index_row
    # Get Segment Corresponding to Min MSE
    idx = raw_mse.loc[
    (raw_mse['input']==var_spec)&\
    (raw_mse['clustered']==False)].index#[err_type].idxmin()

    all_series = pd.DataFrame()

    best_var_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
        regex='policy').iloc[0].astype(float).apply(np.abs).idxmax()

#     &(raw_coef['segmentation']==var_spec[:-2])
    best_coef_all = raw_coef.loc[(raw_coef['inputs']==var_spec)].filter(
        regex='policy').iloc[0].astype(float)[best_var_all]
    best_var = {}
    best_coef = {}
    best_of = 5

    best_c = raw_coef.loc[
    (raw_coef['inputs']==var_spec)&\
    (raw_coef['model']=='Lasso_' + var_spec)].filter(
    regex='policy').iloc[0].astype(float).apply(np.abs).nlargest(best_of)
#             best_t = best_c.apply(lambda x: x.index.tolist(), axis=1)
#             print (best_t.as_matrix())
#             best_coefs =  best_c.as_matrix()
    best_vars = best_c.index.tolist()
    best_coefs = raw_coef.loc[
    (raw_coef['inputs']==var_spec)&\
    (raw_coef['model']=='Lasso_' + var_spec)].iloc[0][best_vars].as_matrix()
    print (best_coefs)

    row = []
    index_row =[]
    for ix, t, i in zip(range(best_of), best_vars, best_coefs):
        row += [
            t.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
            np.round(i,3)
        ]
        index_row += ['Var ' + str(ix), 'Coef ' +  str(ix)]
#             except:
#                 print (var_spec, best_segment, k)
    series = pd.Series([
        year,
        outcome.replace('_',' ').title(),
        'LASSO + '+model_names[var_spec],
        best_var_all.replace('___ethiopia_'+str(year), '').replace('_',' ').title(),
        np.round(best_coef_all,3),
        ''] + row,index=[
        'Year',
        'Outcome',
        'Model',
        'Most Relevant Input',
        'Coef. Most Relevant Input',
        'Household Segmentation']
            + index_row)
    all_series = pd.concat([all_series, series], 1)
        
    return all_series
        

In [9]:
# Run this after all the ones above to generate best coefficient for all cluster's regressions.
import math
all_mse  = pd.DataFrame()
all_coef = pd.DataFrame()
all_raw_mse = pd.DataFrame()

all_raw_coef = pd.DataFrame()
years = [2015]

for outcome in outcomes:
    
    for year in years:
        
        try:
            raw_mse = pd.read_csv(
            path_to_results+outcome+'___'+'output'+ '___'+country+'_'+str(year)+'.csv',
            usecols=['clustered','input',err_type,'segment'])
        except:
            continue
        
        raw_mse['output'] = outcome
        all_raw_mse = pd.concat([all_raw_mse, raw_mse], 0)
        all_raw_mse['segment'] = all_raw_mse['segment'].fillna('').apply(lambda x: x.replace('___ethiopia_'+str(year), ''))
        raw_coef = pd.read_csv(
        path_to_results+'coef_'+outcome+'___'+'output'+'___'+country+'_'+str(year)+'.csv',index_col=0) # 
        
        # Variables On Which Segmentation is Performed
        raw_coef['segmentation'] = raw_coef['model'].apply(
        lambda x:x.replace('Lasso_','').replace('OrdinaryLeastSquares_','').replace('clustered_','').replace('segmented_','').replace('___' + country + '_'+str(year), '')[:-2])

        # Segment Index
        raw_coef['output'] = outcome
        raw_coef['year'] = year
        raw_coef['segment'] = raw_coef['model'].apply(
        lambda x:x.split('_')[-1])
        all_raw_coef = pd.concat([all_raw_coef, raw_coef], 0)

        for var_spec in sorted(model_names):
#             try:
            all_coef = pd.concat([all_coef, summary_coef(year,outcome,var_spec)],1)
            all_coef = pd.concat([all_coef, summary_coef_unsegmented(year,outcome,var_spec)],1)
#             except:
#                 continue
            
all_coef = all_coef.T.set_index(['Outcome','Year','Model'])[[
'Household Segmentation',
'Most Relevant Input', 
'Coef. Most Relevant Input',] + index_row].copy()

all_coef.to_excel(path_to_tables+'coef_'+country+'.xlsx', engine='openpyxl')

[0.50592935186034849 0.22090958350186107 -0.13151397097658268
 0.09686033819672886 0.034812101451402025]
[0.33107150316344958 -0.061361851112130272 0.032029195965863601
 0.029237310273715369 -0.0121394667680632]
[0.13457956986454234 0.12926806541570138 0.074988154173560972
 -0.065394989867510397 0.062720647743362071]
[0.2707375426026073 0.14301820088875156 0.12228592909323548
 0.08100179152171938 -0.054003547635779615]
[0.22376242886955716 0.08123096646325538 0.074787613779306331
 0.071868076791370236 0.0085716382544348997]
[0.71884062476229793 0.33003572907098228 0.14440533776513659
 -0.14070859056137719 0.057490675468969361]
[0.33028448041125325 -0.12452223395533445 0.051365141413611086
 0.040686514464659533 -0.040335192166945033]
[0.14753868166025458 0.12387855520822308 0.11674574216462867
 0.075991419305217037 0.028929222739060494]
[0.27527434553967067 0.1996140203693684 0.14907561574142422
 0.088043804961170369 -0.070465835881261846]
[0.26089470885929811 0.11365226811389313 0.0821